In [1]:
#https://www.kaggle.com/code/sumi25/understand-arima-and-tune-p-d-q/notebook
#Dr. Festus Elleh

In [2]:
#step by step from the coding Elleh video
#Import the libraries

import os                                                     #view operating system file information
import pandas as pd                                           #for dataframe and data manipulations
import numpy as np                                            #Provides array objects for calculations
from numpy import cumsum                  
import matplotlib.pyplot as plt                               #For plots
from statsmodels.tsa.seasonal import seasonal_decompose       #import seasonal decompose
from scipy import signal                                      #For periodogram
from datetime import datetime                                #For datetime
from statsmodels.tsa.stattools import adfuller                #Import augmented dicky-fuller test function
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf #Plot ACF and PACF
from pandas.plotting import autocorrelation_plot              #Look for seasonalality in the data
import pmdarima as pm                                         #for auto-arima

from statsmodels.tsa.arima_model import ARIMA                 #To perform ARIMA analysis
from statsmodels.tsa.statespace.sarimax import SARIMAX        #to perform seasonal ARIMA analysis
import warnings                                               #to ignore warning
warnings.filterwarnings('ignore')
import joblib                                                 #to save and load the model

In [3]:
#shows matplotlib output inline in the jupyter notebook
%matplotlib inline

In [4]:
md = pd.read_csv('medical_time_series .csv')

FileNotFoundError: [Errno 2] No such file or directory: 'medical_time_series .csv'

In [ ]:
#exploratory data analysis (EDA)
#View shape of the data
md.shape

In [ ]:
md.describe()

In [ ]:
#view the data
md.head(10)

In [ ]:
#look for null or missing values
md.isnull().any()

In [ ]:
#Convert Day to Date as time series model needs a date structure
md['Date'] = (pd.date_range(start=datetime(2019, 1, 1),
                           periods=md.shape[0], freq='24H'))
#Set the Date as an index
md.set_index('Date', inplace=True)
md

In [ ]:
#Visualize the data
plt.figure(figsize=(10,5))
plt.plot(md.Revenue)
plt.title('Revenue')
plt.xlabel('Day')
plt.ylabel('Revenue in million $')
plt.grid(True)
plt.show

In [ ]:
#Data cleaning
#Drop null columns
md = md.dropna()

In [ ]:
#Export clean data to csv file
pd.DataFrame(md).to_csv('time_series_clean.csv')

In [ ]:
#Make time series stationary
#Test for stationarity

#identifying whether a time series, is stationary or non-stationary is very important.
result = adfuller(md['Revenue'])

print('Test statistics', result[0])
print('p-value:',result[1])
print('Critical values:', result[4])

In [ ]:
#HypO: Time series is non-stationary
#Hyp1: Time series is stationary

if result[1]<= 0.05:
    print('Reject null hypothesis, the time series is stationary')
else:
    print('Fail to reject the null hypothesis, the time series is non-stationary')

In [ ]:
#Make the time series stationary
md_stationary = md.diff(1).dropna()

#View the data
md_stationary.head(5)

In [ ]:
#Test for stationary data again
result = adfuller(md_stationary['Revenue'])

print('Test statistics', result[0])
print('p-value:',result[1])
print('Critical values:', result[4])

if result[1]<= 0.05:
    print('Reject null hypothesis, the time series is stationary')
else:
    print('Fail to reject the null hypothesis, the time series is non-stationary')

In [ ]:
#Split to train and test
X_train = md_stationary.loc[:'2020-09-30']
X_test = md_stationary.loc[:'2020-10-01']

print('X_train Shape', X_train.shape)
print('X_test Shape', X_test.shape)

#We can see some cyclicality (repeating pattern but no fixed period)

In [ ]:
#Spectral Density 
f, Pxx_den = signal.periodogram(md_stationary['Revenue'])
plt.semilogy(f, Pxx_den)
plt.ylim([1e-6, 1e2])
plt.title('Spectral Density')
plt.xlabel('Frequency')
plt.ylabel('Spectal Density')
plt.show()

In [ ]:
#looking for seasonality in data

#View subset of data to indentify seasonality
#Same seasonality is seen in the data

md.loc[:'2019-06-30'].plot()

In [ ]:
#Continue to look for seasonality in the data
plt.rcParams.update({'figure.figsize':(5,3), 'figure.dpi':120})
autocorrelation_plot(md.Revenue.tolist())

In [ ]:
#Perform decomposition
########
decomp = seasonal_decompose(md_stationary.Revenue, period=90)

#Plot decomp
decomp.plot()
plt.show()

#Some seasonality is seen in the data as shown below

In [ ]:
#Plot seasonality

plt.title('Seasonality')
decomp.seasonal.plot()

In [ ]:
#View the trend
plt.title('Trend')
decomp.trend.plot()

In [ ]:
#Plot the residual
plt.title('Residuals')
decomp.resid.plot()

In [ ]:
autocorrelation_plot(md_stationary)
plt.show()

In [ ]:
#change array size to 1d
array_fix = md_stationary['Revenue']

In [ ]:
#Plot the ACF of md
plot_acf(array_fix)
plt.show()

In [ ]:
#partial autocorrelation
plot_pacf(array_fix)
plt.show()

In [ ]:
#Pick best order by AIC. Smaller value for AIC is preferable_aic = np.inf

best_aic = np.inf
best_order = None
bestmdl = None
rng = range(3)
for p in rng: #loop over p values
    for q in rng: #loop over q values
        try:
            
            #create and fit ARIMA(p,q) model
            model = SARIMAX(array_fix, order=(p,1,q), trend='c')
            results = model.fit()
            tmp_aic = results.aic
            print(p, q, results.aic, results.bic)
            if tmp_aic < best_aic:
                best_aic = tmp_aic
                best_order = (p,q)
                best_mdl = tmp_mdl
        #Print order and results
        except:
            print(p,q, None, None)
            
print('nBest AIC: {:6.5f} | order: {}'.format(best_aic, best_order))

In [ ]:
#Auto ARIMA

#Find the best model using Auto-ARIMA. (Includes seasonality)
model = pm.auto_arima(array_fix,
                     seasonal=True, m=90,
                     d=1, D=1,
                     start_p=1, start_q=1,
                     max_p=2, max_q=2,
                     max_P=2, max_Q=2,
                     trace=True,
                     error_action='ignore',
                     suppress_warnings=True)

print(model.summary())   #Print model summary

In [ ]:
#Time series modeling

#Create the time series model

model = SARIMAX(array_fix, order=(1,1,0), seasonal_order=(1,1,0,90))
results = model.fit()
results.summary()

In [ ]:
#Print mean absolute error
mae = np.mean(np.abs(results.resid))
print('Mean absolute error', mae)

In [ ]:
#Create the 4 diagnostics plots
results.plot_diagnostics().show()

In [ ]:
#Validate with test set

#Generate predictions
prediction = results.get_prediction(start=-90)

#Extract prediction mean
mean_prediction = prediction.predicted_mean
#Get confidence intervals of predictions
confidence_intervals = prediction.conf_int()
#Select lower and upper confidence limits
lower_limits = confidence_intervals.loc[:,'lower Revenue']
upper_limits = confidence_intervals.loc[:, 'upper Revenue']
#Print best estimate predictions
print(mean_prediction)

In [ ]:
#Plot the data
plt.figure(figsize=(12,4))
plt.plot(X_test.index, X_test, label='observed (test set)')

#plot your mean predictions
plt.plot(mean_prediction.index, mean_prediction, color='r', label='forecast')

#shade the area between your confidence limits
plt.fill_between(lower_limits.index, lower_limits, upper_limits, color='pink')

#set labels, legends and show plot
plt.title('Forecast comparing with test data')
plt.xlabel('Date')
plt.ylabel('Revenue (in millions)')
plt.legend()
plt.show()

In [ ]:
#Perform Forecast

#forecast
diff_forecast = results.get_forecast(steps=180)
mean_forecast = diff_forecast.predicted_mean
print(mean_forecast.head(5))
#Get confidence intervals of predictions
confidence_intervals = diff_forecast.conf_int()
#Select lower and upper confidence limits
lower_limits = confidence_intervals.loc[:,'lower Revenue']
upper_limits = confidence_intervals.loc[:,'upper Revenue']

In [ ]:
#Plot the forecasted revenue data
plt.plot(array_fix.index, array_fix, label='Observed')
#Plot the mean predictions
plt.plot(mean_forecast.index, mean_forecast, color='r', label='Forecast')

#Shade the area between the confidence limits
plt.fill_between(lower_limits.index, lower_limits, upper_limits, color='pink')

#set labels, legends and show plot
plt.title('Revenue with forecasted projections for 2021')
plt.xlabel('Date')
plt.ylabel('Revenue (in millions)')
#plt.xtick(rotation = 45)
plt.legend()
plt.show()

In [ ]:
plt.title('Revenue projection for year 2021')
plt.xlabel('Date')
plt.ylabel('Revenue (in millions)')
mean_forecast.plot()

In [ ]:
#Saving the model

joblib.dump(model, 'med_timeseries.pkl')

In [ ]:
loaded_model = joblib.load('med_timeseries.pkl')